In [ ]:
# I use qiskit==0.25.4 here
import qiskit
print(qiskit.__qiskit_version__)

##############
# {'qiskit-terra': '0.17.2', 
#  'qiskit-aer': '0.8.2', 
#  'qiskit-ignis': '0.6.0', 
#  'qiskit-ibmq-provider': '0.12.3', 
#  'qiskit-aqua': '0.9.1', 
#  'qiskit': '0.25.4', 
#  'qiskit-nature': None, 
#  'qiskit-finance': None, 
#  'qiskit-optimization': None, 
#  'qiskit-machine-learning': None}
##############

# Ignore warnings for a clear view
import warnings
warnings.filterwarnings('ignore')

# Packages for constructing Hamiltonian
from qiskit.chemistry.drivers import PySCFDriver, UnitsType
from qiskit.chemistry import FermionicOperator, MP2Info
from qiskit.chemistry import FermionicOperator
from qiskit.chemistry.drivers import HFMethodType
from groupedFermionicOperator import groupedFermionicOperator
from qiskit.aqua.algorithms import NumPyEigensolver
import re

# Math and visualization tools
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

# Packages for VQE
from qiskit.aqua.components.optimizers import COBYLA, SPSA, SLSQP, AQGD
from qiskit.circuit.library import RealAmplitudes, EfficientSU2
from qiskit.aqua.algorithms import VQE
from qiskit import Aer, QuantumCircuit
from qiskit.circuit import Parameter
from qiskit.aqua import QuantumInstance

# Monitor runtime
import time

# Molecule Coordinates

In [ ]:
with open('Ace_enol_b3lyp_optimized.txt') as f:
    lines = f.readlines()
lines

In [ ]:
mol_coord = ""
for line in lines:
    atom = re.search(r'[A-Z]', line).group(0)
    coords = re.findall(r'-?\d\.\d+', line)
    mol_coord += atom
    for coord in coords:
        mol_coord += " " + coord
    mol_coord += ";"
mol_coord = mol_coord[:-1]
print(mol_coord)

In [ ]:
driver = PySCFDriver(atom=mol_coord, unit=UnitsType.ANGSTROM, 
                     charge=0, spin=0, basis='sto3g')
molecule = driver.run()
print("# of MO:", molecule.num_orbitals)
hf_energy = molecule.hf_energy
print("HF Energy:", hf_energy)

# Get Hamiltonian

In [ ]:
def get_qubit_op_QEE():
    driver = PySCFDriver(atom=mol_coord, unit=UnitsType.ANGSTROM, 
                         charge=0, spin=0, basis='sto3g')
    molecule = driver.run()
    freeze_list = [i for i in range(14)]
    remove_list = [i for i in range(20, 26)]
    repulsion_energy = molecule.nuclear_repulsion_energy
    num_particles = molecule.num_alpha + molecule.num_beta
    num_spin_orbitals = molecule.num_orbitals * 2
    remove_list = [x % molecule.num_orbitals for x in remove_list]
    freeze_list = [x % molecule.num_orbitals for x in freeze_list]
    remove_list = [x - len(freeze_list) for x in remove_list]
    remove_list += [x + molecule.num_orbitals - len(freeze_list)  for x in remove_list]
    freeze_list += [x + molecule.num_orbitals for x in freeze_list]
    ferOp = FermionicOperator(h1=molecule.one_body_integrals, h2=molecule.two_body_integrals)
    ferOp, energy_shift = ferOp.fermion_mode_freezing(freeze_list)
    num_spin_orbitals -= len(freeze_list)
    num_particles -= len(freeze_list)
    ferOp = ferOp.fermion_mode_elimination(remove_list)
    num_spin_orbitals -= len(remove_list)
    shift = energy_shift + repulsion_energy
    g = groupedFermionicOperator(ferOp, num_particles, mode='rhf')
    qubitOp = g.to_paulis(10)
    return qubitOp, num_particles, num_spin_orbitals, shift

In [ ]:
# starting time
start = time.time()

# program body starts
qubitOp_QEE, num_particles_QEE, num_spin_orbitals_QEE, shift_QEE = get_qubit_op_QEE()
print(qubitOp_QEE, num_particles_QEE, num_spin_orbitals_QEE, shift_QEE)
# program body ends

# end time
end = time.time()

# total time taken
print(f"Runtime of the program is {end - start}")

In [ ]:
exact_result = NumPyEigensolver(qubitOp_QEE).run()
exact_energy = np.real(exact_result.eigenvalues)[0] + shift_QEE
print("Exact Energy for QEE Hamiltonian", exact_energy)

In [ ]:
map_type = 'jordan_wigner'
def get_qubit_op_JW():
    driver = PySCFDriver(atom=mol_coord, unit=UnitsType.ANGSTROM, 
                         charge=0, spin=0, basis='sto3g')
    molecule = driver.run()
    freeze_list = [i for i in range(14)]
    remove_list = [i for i in range(20, 26)]
    repulsion_energy = molecule.nuclear_repulsion_energy
    num_particles = molecule.num_alpha + molecule.num_beta
    num_spin_orbitals = molecule.num_orbitals * 2
    remove_list = [x % molecule.num_orbitals for x in remove_list]
    freeze_list = [x % molecule.num_orbitals for x in freeze_list]
    remove_list = [x - len(freeze_list) for x in remove_list]
    remove_list += [x + molecule.num_orbitals - len(freeze_list)  for x in remove_list]
    freeze_list += [x + molecule.num_orbitals for x in freeze_list]
    ferOp = FermionicOperator(h1=molecule.one_body_integrals, h2=molecule.two_body_integrals)
    ferOp, energy_shift = ferOp.fermion_mode_freezing(freeze_list)
    num_spin_orbitals -= len(freeze_list)
    num_particles -= len(freeze_list)
    ferOp = ferOp.fermion_mode_elimination(remove_list)
    num_spin_orbitals -= len(remove_list)
    shift = energy_shift + repulsion_energy
    qubitOp = ferOp.mapping(map_type=map_type, threshold=0.00000001)
    return qubitOp, num_particles, num_spin_orbitals, shift

In [ ]:
# starting time
start = time.time()

# program body starts
qubitOp, num_particles, num_spin_orbitals, shift = get_qubit_op_JW()
print(qubitOp, num_particles, num_spin_orbitals, shift)
# program body ends

# end time
end = time.time()

# total time taken
print(f"Runtime of the program is {end - start}")

In [ ]:
exact_result_JW = NumPyEigensolver(qubitOp).run()
exact_energy_JW = np.real(exact_result_JW.eigenvalues)[0] + shift
print("Exact Energy for JW Hamiltonian", exact_energy_JW)

# Define Ansatz

In [ ]:
num_circ = 100
ry_entangle = []
for i in range(num_circ):
    qc = QuantumCircuit(2)
    theta_list = []
    for j in range(2):
        theta = Parameter('θ' + str(2*i + j))
        theta_list.append(theta)
    qc.ry(theta_list[0], 0)
    qc.ry(theta_list[1], 1)
    qc.cx(0, 1)
    # qc.draw("mpl");
    gate = qc.to_gate(label="RyE" + str(i))
    ry_entangle.append(gate)
qc.draw("mpl");

In [ ]:
ansatz = QuantumCircuit(8)
# 1
ansatz.append(ry_entangle[0], [0, 1])
ansatz.append(ry_entangle[1], [2, 3])
ansatz.append(ry_entangle[2], [4, 5])
ansatz.append(ry_entangle[3], [6, 7])
# 2
ansatz.append(ry_entangle[4], [1, 2])
ansatz.append(ry_entangle[5], [3, 4])
ansatz.append(ry_entangle[6], [5, 6])
ansatz.append(ry_entangle[7], [7, 0])
# 3
ansatz.append(ry_entangle[8], [0, 1])
ansatz.append(ry_entangle[9], [2, 3])
ansatz.append(ry_entangle[10], [4, 5])
ansatz.append(ry_entangle[11], [6, 7])
# 4
ansatz.append(ry_entangle[12], [1, 2])
ansatz.append(ry_entangle[13], [3, 4])
ansatz.append(ry_entangle[14], [5, 6])
ansatz.append(ry_entangle[15], [7, 0])
# 5
ansatz.append(ry_entangle[16], [0, 1])
ansatz.append(ry_entangle[17], [2, 3])
ansatz.append(ry_entangle[18], [4, 5])
ansatz.append(ry_entangle[19], [6, 7])
# 6
ansatz.append(ry_entangle[20], [1, 2])
ansatz.append(ry_entangle[21], [3, 4])
ansatz.append(ry_entangle[22], [5, 6])
ansatz.append(ry_entangle[23], [7, 0])
# 7
ansatz.append(ry_entangle[24], [0, 1])
ansatz.append(ry_entangle[25], [2, 3])
ansatz.append(ry_entangle[26], [4, 5])
ansatz.append(ry_entangle[27], [6, 7])
# 8
ansatz.append(ry_entangle[28], [1, 2])
ansatz.append(ry_entangle[29], [3, 4])
ansatz.append(ry_entangle[30], [5, 6])
ansatz.append(ry_entangle[31], [7, 0])
# 9
ansatz.append(ry_entangle[32], [0, 1])
ansatz.append(ry_entangle[33], [2, 3])
ansatz.append(ry_entangle[34], [4, 5])
ansatz.append(ry_entangle[35], [6, 7])
# 10
ansatz.append(ry_entangle[36], [1, 2])
ansatz.append(ry_entangle[37], [3, 4])
ansatz.append(ry_entangle[38], [5, 6])
ansatz.append(ry_entangle[39], [7, 0])
# 11
ansatz.append(ry_entangle[40], [0, 1])
ansatz.append(ry_entangle[41], [2, 3])
ansatz.append(ry_entangle[42], [4, 5])
ansatz.append(ry_entangle[43], [6, 7])
# 12
ansatz.append(ry_entangle[44], [1, 2])
ansatz.append(ry_entangle[45], [3, 4])
ansatz.append(ry_entangle[46], [5, 6])
ansatz.append(ry_entangle[47], [7, 0])
# 13
ansatz.append(ry_entangle[48], [0, 1])
ansatz.append(ry_entangle[49], [2, 3])
ansatz.append(ry_entangle[50], [4, 5])
ansatz.append(ry_entangle[51], [6, 7])
# 14
ansatz.append(ry_entangle[52], [1, 2])
ansatz.append(ry_entangle[53], [3, 4])
ansatz.append(ry_entangle[54], [5, 6])
ansatz.append(ry_entangle[55], [7, 0])
# 15
ansatz.append(ry_entangle[56], [0, 1])
ansatz.append(ry_entangle[57], [2, 3])
ansatz.append(ry_entangle[58], [4, 5])
ansatz.append(ry_entangle[59], [6, 7])
# 16
ansatz.append(ry_entangle[60], [1, 2])
ansatz.append(ry_entangle[61], [3, 4])
ansatz.append(ry_entangle[62], [5, 6])
ansatz.append(ry_entangle[63], [7, 0])
# 17
ansatz.append(ry_entangle[64], [0, 1])
ansatz.append(ry_entangle[65], [2, 3])
ansatz.append(ry_entangle[66], [4, 5])
ansatz.append(ry_entangle[67], [6, 7])
# 18
ansatz.append(ry_entangle[68], [1, 2])
ansatz.append(ry_entangle[69], [3, 4])
ansatz.append(ry_entangle[70], [5, 6])
ansatz.append(ry_entangle[71], [7, 0])
# 19
ansatz.append(ry_entangle[72], [0, 1])
ansatz.append(ry_entangle[73], [2, 3])
ansatz.append(ry_entangle[74], [4, 5])
ansatz.append(ry_entangle[75], [6, 7])
# 20
ansatz.append(ry_entangle[76], [1, 2])
ansatz.append(ry_entangle[77], [3, 4])
ansatz.append(ry_entangle[78], [5, 6])
ansatz.append(ry_entangle[79], [7, 0])
# # 21
# ansatz.append(ry_entangle[80], [0, 1])
# ansatz.append(ry_entangle[81], [2, 3])
# ansatz.append(ry_entangle[82], [4, 5])
# ansatz.append(ry_entangle[83], [6, 7])
# # 22
# ansatz.append(ry_entangle[84], [1, 2])
# ansatz.append(ry_entangle[85], [3, 4])
# ansatz.append(ry_entangle[86], [5, 6])
# ansatz.append(ry_entangle[87], [7, 0])
# # 23
# ansatz.append(ry_entangle[88], [0, 1])
# ansatz.append(ry_entangle[89], [2, 3])
# ansatz.append(ry_entangle[90], [4, 5])
# ansatz.append(ry_entangle[91], [6, 7])
# # 24
# ansatz.append(ry_entangle[92], [1, 2])
# ansatz.append(ry_entangle[93], [3, 4])
# ansatz.append(ry_entangle[94], [5, 6])
# ansatz.append(ry_entangle[95], [7, 0])
# # 25
# ansatz.append(ry_entangle[96], [0, 1])
# ansatz.append(ry_entangle[97], [2, 3])
# ansatz.append(ry_entangle[98], [4, 5])
# ansatz.append(ry_entangle[99], [6, 7])

# last rotation
num_layers = 20
for j in range(8):
    theta = Parameter('θ' + str((num_layers) * 8 + j))
    ansatz.ry(theta, j)

ansatz.draw("mpl");

# Run Simulation

In [ ]:
iterations = 50000
num_shots = 1024
# optimizer = COBYLA(maxiter=iterations)
# optimizer = SPSA(maxiter=iterations)
optimizer = SLSQP(maxiter=iterations)
# optimizer = AQGD(maxiter=iterations)
backend = Aer.get_backend('statevector_simulator')

In [ ]:
# VQE Error (iterations) Circular
# 5 layers (147): 18.076119431096416
# 6 layers (228): 18.135864979192817
# 7 layers (195): 18.047517485344002
# 8 layers (292): 18.13185221848407
# 9 layers (244): 18.04669784455548
# 10 layers (1338): 15.089220833715302
# 11 layers (292): 18.046631550355983
# 12 layers (1683): 15.087343441627326
# 13 layers (453): 18.046248014280835
# 14 layers (25906): 0.3331261860567736
# 15 layers (18972): 0.34328656153050474
# 16 layers (22209): 0.2967593388852663
# 17 layers (582): 18.045189176870693
# 18 layers (20364): 0.3757898168307605
# 19 layers (647): 18.045050609788973
# 20 layers (28240): 0.2671464361201714
# 21 layers (28690): 0.28519471876222724
# 22 layers (34238): 0.27973714605806776
# 23 layers (32641): 0.20873089102686038
# 24 layers (31979): 0.23827525091405732
# 25 layers (31165): 0.1643961447482667

In [ ]:
# Ansatz circuit initial parameters for rotations
initial_point = [0.]*ansatz.num_parameters

# Store optimization results
counts = []
values = []
def store_intermediate_result(eval_count, parameters, mean, std):
    counts.append(eval_count)
    values.append(mean)
# Run VQE
seed = 0 # For consistent results on simulator
qi = QuantumInstance(backend=backend, shots=num_shots, seed_simulator=seed)
vqe = VQE(var_form=ansatz, optimizer=optimizer, callback=store_intermediate_result, 
          quantum_instance=qi, initial_point=initial_point)
result = vqe.compute_minimum_eigenvalue(operator=qubitOp_QEE)
vqe_energy = result.eigenvalue.real + shift_QEE
print("Exact energy:", exact_energy, "Ha")
print("VQE energy:", vqe_energy, "Ha")
print("VQE Error:", abs(exact_energy - vqe_energy) * 627.5, "kcal/mol")
print("HF energy:", hf_energy, "Ha")
print("HF Error:", abs(exact_energy - hf_energy) * 627.5, "kcal/mol")
print("Evaluation counts:", result['cost_function_evals'])
print("--------------------------------------------------")